**------------------------------------------------------------------------------------------------------------------------------------------------------**

**Input: Drug Repurposing Knowledge Graph (DRKG)**

**Purpose: Return trained GraphSAGE, GCN and GAT on DRKG**

**------------------------------------------------------------------------------------------------------------------------------------------------------**

# Librairies

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import json

import os
import re
import random
import itertools
import import_ipynb

import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F

import dgl
import dgl.nn as dglnn
import dgl.function as fn
from dgl.nn import HeteroGraphConv, SAGEConv, GraphConv, GATConv
from dgl.data.utils import save_graphs, load_graphs

from torch_geometric.explain import characterization_score

from captum.attr import Saliency, IntegratedGradients

from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

from functools import partial

import warnings
warnings.simplefilter("ignore")

In [7]:
pip install captum

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 18.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910475 sha256=de8d3d3a6fb4fccf023a349149c4ec8a0b3cd5d7ad5bc09e65ba90c9161bd46c
  Stored in directory: /tmp/pip-ephem-wheel-cache-w7t5osw4/wheels/ff/6c/d6/f131acff9176ff91cb7ce97ddcd7170182c99533bf31c86a1d
Successfully built torch_geometric

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install dgl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**Parameters: define GNN variant, # features per node, # epochs to train the network**

In [9]:
# gnn_variant = 'GAT'
# gnn_variant = 'GraphSAGE'
gnn_variant = 'GCN'

In [10]:
n_node_features = 10

In [11]:
n_epochs = 300

# 1) Create Heterogeneous Graph/Knowledge Graph

**Get Drug Repurposing Knowledge Graph (DRKG)**

In [13]:
df = pd.read_csv('./Input/drkg.tsv', sep='\t', header=None)

FileNotFoundError: [Errno 2] No such file or directory: './Input/drkg.tsv'

In [6]:
# Only consider Drug + Disease
df = df[(df[0].str.startswith('Compound') | df[0].str.startswith('Disease')) & (df[2].str.startswith('Compound') | df[2].str.startswith('Disease'))]

In [7]:
df = df.dropna()

In [8]:
triplets = df.values.tolist()

**Assign an ID to each node/entity: create a dictionary of node-types -> each dictionary further consists of a dictionary mapping a node to an ID**

In [9]:
entity_dictionary = {}
def insert_entry(entry, ent_type, dic):
    if ent_type not in dic:
        dic[ent_type] = {}
    ent_n_id = len(dic[ent_type])
    if entry not in dic[ent_type]:
         dic[ent_type][entry] = ent_n_id
    return dic

for triple in triplets:
    src = str(triple[0])
    split_src = src.split('::')
    src_type = split_src[0]
    dest = str(triple[2])
    split_dest = dest.split('::')
    dest_type = split_dest[0]
    insert_entry(src, src_type, entity_dictionary)
    insert_entry(dest, dest_type, entity_dictionary)

**Create a dictionary of relations: the key is the relation and the value is a list of (source node ID, destination node ID) tuples**

In [10]:
edge_dictionary = {}
for triple in triplets:
    src = str(triple[0])
    split_src = src.split('::')
    src_type = split_src[0]
    dest = str(triple[2])
    split_dest = dest.split('::')
    dest_type = split_dest[0]
    
    src_int_id = entity_dictionary[src_type][src]
    dest_int_id = entity_dictionary[dest_type][dest]
    
    pair = (src_int_id, dest_int_id)
    etype = (src_type, triple[1], dest_type)
    if etype in edge_dictionary:
        edge_dictionary[etype] += [pair]
    else:
        edge_dictionary[etype] = [pair]

**Create HeteroGraph**

In [11]:
g = dgl.heterograph(edge_dictionary)

**Add some synthetic/random node features**

In [12]:
node_features = {}
for ntype in g.ntypes:
    g.nodes[ntype].data['h'] = torch.randn(g.num_nodes(ntype), n_node_features).requires_grad_(True)
    node_features[ntype] = g.nodes[ntype].data['h']

**Define etype: we only want to predict one edge type**

In [13]:
etype = ('Compound', 'DRUGBANK::treats::Compound:Disease', 'Disease')

# 2) Define Graph Neural Network (GNN)

In [14]:
def construct_negative_graph(graph, k, etype):
    utype, _, vtype = etype
    src, dst = graph.edges(etype=etype)
    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.num_nodes(vtype), (len(src) * k,))
    return dgl.heterograph({etype: (neg_src, neg_dst)}, num_nodes_dict = {ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

In [15]:
class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [16]:
class Net(torch.nn.Module):
    def __init__(self, etypes):
        super().__init__()
        if gnn_variant == 'GraphSAGE':
            self.conv1 = HeteroGraphConv({etype: SAGEConv(10, 10, 'mean') for etype in etypes})
            self.conv2 = HeteroGraphConv({etype: SAGEConv(10, 10, 'mean') for etype in etypes})
            self.pred = HeteroDotProductPredictor()
        elif gnn_variant == 'GCN':
            self.conv1 = HeteroGraphConv({etype: GraphConv(10, 10) for etype in etypes})
            self.conv2 = HeteroGraphConv({etype: GraphConv(10, 10) for etype in etypes})
            self.pred = HeteroDotProductPredictor()
        elif gnn_variant == 'GAT':
            self.conv1 = HeteroGraphConv({etype: GATConv(10, 10, 1) for etype in etypes})
            self.conv2 = HeteroGraphConv({etype: GATConv(10, 10, 1) for etype in etypes})
            self.pred = HeteroDotProductPredictor()
        else:
            print('No model has been chosen !')
        
    def forward(self, pos_g, neg_g, node_features, etype, edge_weight=None):
        if edge_weight is None:
            h = self.conv2(pos_g, self.conv1(pos_g, node_features))
        else:
            h = self.conv2(pos_g, self.conv1(pos_g, node_features, mod_kwargs={etype:{'edge_weight': edge_weight[etype]} for etype in g.etypes}))
        return self.pred(pos_g, h, etype), self.pred(neg_g, h, etype)

# 3) Train Model

**Initialize model and optimizer**

In [33]:
model = Net(g.etypes)
opt = torch.optim.Adam(model.parameters())

**Train/test split**

In [18]:
eids = np.arange(g.number_of_edges(etype))
eids = np.random.permutation(eids)
eids = torch.tensor(eids, dtype=torch.int64)

train_size = int(len(eids) * 0.8)
test_size = g.number_of_edges(etype) - train_size

train_indices = {etype: eids[test_size:]}
test_indices = {etype: eids[:test_size]}

In [19]:
g_train = dgl.remove_edges(g, eids=test_indices[etype], etype=etype)
g_test = dgl.remove_edges(g, eids=train_indices[etype], etype=etype)

**Define metrics**

In [20]:
def compute_loss(pos_score, neg_score):
    n_edges = pos_score.shape[0]
    return (1 - pos_score + neg_score.view(n_edges, -1)).clamp(min=0).mean()

In [21]:
def compute_auroc(pos_score, neg_score): 
    model.eval()
    scores = torch.cat([pos_score, neg_score]).view(-1)
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return roc_auc_score(labels.detach().numpy(), scores.detach().numpy())

def eval_model(pos_score, neg_score):
    model.eval()
    scores = torch.cat([pos_score, neg_score]).view(-1).detach().numpy()
    scores = np.where(scores >= 0.6, 1, 0)
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).detach().numpy()
    return precision_score(labels, scores), recall_score(labels, scores)

**Train model**

In [34]:
for epoch in range(n_epochs+1): 
    # forward
    g_neg_train = construct_negative_graph(g_train, 5, etype)
    pos_score, neg_score = model(g_train, g_neg_train, node_features, etype)

    # compute loss, auroc
    loss = compute_loss(pos_score, neg_score)
    auroc = compute_auroc(pos_score, neg_score)

    # backward
    opt.zero_grad()
    loss.backward()
    opt.step()

    # print epoch + auroc
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, AUROC: {auroc:.4f}')

Epoch: 000, AUROC: 0.4934
Epoch: 010, AUROC: 0.6074
Epoch: 020, AUROC: 0.7299
Epoch: 030, AUROC: 0.8124
Epoch: 040, AUROC: 0.8490
Epoch: 050, AUROC: 0.8602
Epoch: 060, AUROC: 0.8723
Epoch: 070, AUROC: 0.8876
Epoch: 080, AUROC: 0.8955
Epoch: 090, AUROC: 0.9031
Epoch: 100, AUROC: 0.9031
Epoch: 110, AUROC: 0.9077
Epoch: 120, AUROC: 0.9070
Epoch: 130, AUROC: 0.9069
Epoch: 140, AUROC: 0.9070
Epoch: 150, AUROC: 0.9080
Epoch: 160, AUROC: 0.9080
Epoch: 170, AUROC: 0.9076
Epoch: 180, AUROC: 0.9061
Epoch: 190, AUROC: 0.9087
Epoch: 200, AUROC: 0.9087
Epoch: 210, AUROC: 0.9083
Epoch: 220, AUROC: 0.9094
Epoch: 230, AUROC: 0.9061
Epoch: 240, AUROC: 0.9107
Epoch: 250, AUROC: 0.9086
Epoch: 260, AUROC: 0.9076
Epoch: 270, AUROC: 0.9062
Epoch: 280, AUROC: 0.9092
Epoch: 290, AUROC: 0.9087
Epoch: 300, AUROC: 0.9077


**Test model**

In [35]:
# forward
g_neg_test = construct_negative_graph(g_test, 5, etype)
pos_score, neg_score = model(g_test, g_neg_test, node_features, etype)

# eval model
precision, recall = eval_model(pos_score, neg_score)
auroc = compute_auroc(pos_score, neg_score)

In [36]:
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1_score:.4f}')
print(f'AUROC: {auroc:.4f}')

Precision: 0.7978
Recall: 0.2857
F1-Score: 0.4207
AUROC: 0.9377


**Save model**

In [37]:
torch.save(model, f'GNNModels/{gnn_variant}')